In [1]:
import serial


def open_glove():
    return serial.Serial('COM8')


def close_glove(ser):
    ser.close()


def buzz(ser, loc, lenms=500):
    '''Buzz given location for given duration in ms
    location codes:
    t, b, l, r, a'''
    loc_ar = {'t': '1000', 'r': '0100', 'b': '0010', 'l': '0001', 'a': '1111'}
    loc_code = loc_ar[loc]
    com = loc_code+'buzz'+str(lenms)+'.'
    ser.write(str.encode(com))


def spin(times):
    for i in range(times):
        buzz('t', 50)
        buzz('r', 50)
        buzz('b', 50)
        buzz('l', 50)


def guide(position, target, marginOfError, ser, mode, previousDirection):
    """Will print guiding information to guide the user's hand to the target
    position: tupple with the hand's location
    target: tuple with the target's position
    marginOfError: double representing the allowed distance from target that is considered a success
    ser: connection to the glove
    mode: if the program is guiding via console (0) or wristband (1)
    previousDirection: direction this method instructed last time it was executed
    """
    deltaX = abs(position[0] - target[0])
    deltaY = abs(position[1] - target[1])
    # Guide in the direction of the furthers away first
    # If there is more horizontal distance than vertical distance, move horizontally
    if deltaX > deltaY:
        # Check the x coordinate to guide to the target
        previousDirection = guideHorizontal(
            position[0], target[0], marginOfError, ser, mode, previousDirection)
    else:
        # Check the y coordinate to guide to the target
        previousDirection = guideVertical(
            position[1], target[1], marginOfError, ser, mode, previousDirection)
    return previousDirection


def guideVertical(position, target, marginOfError, ser, mode, previousDirection):
    """Will print guiding information to guide the user's hand to the target
    position: hand's y location
    target: target's y location
    marginOfError: double representing the allowed distance from target that is considered a success
    ser: connection to the glove
    mode: if the program is guiding via console (0) or wristband (1)
    previousDirection: direction this method instructed last time it was executed
    """
    # Check the y coordinate to guide to the target
    if position > target * (1.0 - marginOfError) and not position < target * (1.0 + marginOfError):
        # Only say up if you weren't going up before
        if previousDirection != "up":
            previousDirection = "up"
            if mode == 0:
                print("Up")
            else:
                buzz(ser, "t")
    elif position < target * (1.0 + marginOfError) and not position > target * (1.0 - marginOfError):
        # Only say down if you weren't going down before
        if previousDirection != "down":
            previousDirection = "down"
            if mode == 0:
                print("Down")
            else:
                buzz(ser, "b")
    return previousDirection


def guideHorizontal(position, target, marginOfError, ser, mode, previousDirection):
    """Will print guiding information to guide the user's hand to the target
    position: hand's x location
    target: target's x location
    marginOfError: double representing the allowed distance from target that is considered a success
    ser: connection to the glove
    mode: if the program is guiding via console (0) or wristband (1)
    previousDirection: direction this method instructed last time it was executed
    """
    # Check the x coordinate to guide to the target
    if position > target * (1.0 - marginOfError) and not position < target * (1.0 + marginOfError):
        # Only say right if you weren't going right before
        if previousDirection != "right":
            previousDirection = "right"
            if mode == 0:
                print("Right")
            else:
                buzz(ser, "l")
    elif position < target * (1.0 + marginOfError) and not position > target * (1.0 - marginOfError):
        # Only say left if you weren't going left before
        if previousDirection != "left":
            previousDirection = "left"
            if mode == 0:
                print("Left")
            else:
                buzz(ser, "r")
    return previousDirection
